In [1]:
import numpy as np
import tensorflow as tf

In [2]:
# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="model_231103_01.tflite")
interpreter.allocate_tensors()

In [3]:
print(interpreter.get_input_details()[0])

{'name': 'serving_default_input_1:0', 'index': 0, 'shape': array([1, 7], dtype=int32), 'shape_signature': array([-1,  7], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}


In [4]:
print(interpreter.get_input_details()[1])

{'name': 'serving_default_input_2:0', 'index': 1, 'shape': array([ 1, 10,  1], dtype=int32), 'shape_signature': array([-1, 10,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}


In [ ]:
# 입력 형태 설정 -> tflite 생성 때 미리 명시했으면 안해도 됨
# interpreter.resize_tensor_input(0, (1,7))
# interpreter.allocate_tensors()
# interpreter.resize_tensor_input(1, (1, 10, 1))
# interpreter.allocate_tensors()

2023,11,8,3,10,1,11,69,246
2023,11,8,3,10,1,12,69,246
2023,11,8,3,10,1,13,69,246
2023,11,8,3,10,1,14,69,246
2023,11,8,3,10,1,15,69,246
2023,11,8,3,10,1,16,69,246
2023,11,8,3,10,1,17,69,246
2023,11,8,3,10,1,18,69,246
2023,11,8,3,10,1,19,69,246
2023,11,8,3,10,1,20,69,246

In [17]:
sample_0 = [2023,11,8,3,10,1,20]
sample_0 = np.array(sample_0, dtype = np.float32)
sample_0 = np.reshape(sample_0, (1,7))

In [18]:
sample_1 = [69] * 10
sample_1 = np.array(sample_1, dtype = np.float32)
sample_1 = np.reshape(sample_1, (1,10,1))

In [19]:
interpreter.set_tensor(interpreter.get_input_details()[0]['index'], sample_0)
interpreter.set_tensor(interpreter.get_input_details()[1]['index'], sample_1)
interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(interpreter.get_output_details()[0]['index'])
print(output_data)

[[242.83331]]


---

## C++ 에서 따라하기

```cpp
#include <tensorflow/lite/interpreter.h>
#include <tensorflow/lite/kernels/register.h>
#include <tensorflow/lite/model.h>

int main() {
    // Load the TFLite model and allocate tensors
    std::unique_ptr<tflite::FlatBufferModel> model = tflite::FlatBufferModel::BuildFromFile("model_231103_01.tflite");
    tflite::ops::builtin::BuiltinOpResolver resolver;
    std::unique_ptr<tflite::Interpreter> interpreter;
    tflite::InterpreterBuilder(model.get(), resolver)(&interpreter);
    interpreter->AllocateTensors();

    // Input data setup
    int batch_size = 1;
    int input_seq_length = 10;
    int input_dim = 1; // sensor 갯수
    int output_dim = 1;

    float sample_0[7] = {2023, 11, 8, 3, 10, 1, 20};
    float sample_1[10] = {69, 69, 69, 69, 69, 69, 69, 69, 69, 69};

    // Set input tensors
    interpreter->ResizeInput(0, {batch_size, 7});
    interpreter->ResizeInput(1, {batch_size, 10, 1});
    interpreter->AllocateTensors();

    float* input_data_0 = interpreter->typed_input_tensor<float>(0);
    float* input_data_1 = interpreter->typed_input_tensor<float>(1);

    for (int i = 0; i < 7; i++) {
        input_data_0[i] = sample_0[i];
    }

    for (int i = 0; i < 10; i++) {
        input_data_1[i] = sample_1[i];
    }

    // Invoke inference
    interpreter->Invoke();

    // Get the output tensor
    float* output_data = interpreter->typed_output_tensor<float>(0);

    // Print the output data
    for (int i = 0; i < output_dim; i++) {
        std::cout << "Output[" << i << "] = " << output_data[i] << std::endl;
    }

    return 0;
}

```